This notebook will use IFCB images to train a convolutional neural network and have it classify images.

In [ ]:
import matplotlib.pyplot as plt
%pylab inline
%matplotlib inline


from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, ZeroPadding2D, Input
from keras.layers import concatenate
from keras.preprocessing import image as keras_image
from keras.optimizers import Adam
from keras.backend import tf as ktf
from keras.constraints import maxnorm
from keras.layers import Add, Multiply, Concatenate, Average

import keras.backend as K
import numpy as np
import cv2 as cv2
from collections import Counter
import os
import skimage.transform as ski_transform
import skimage.io as ski_io
from skimage import img_as_float


from PIL import Image as PIL_Image

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report, confusion_matrix

import pandas as pd
import seaborn as sns
from scipy.io import loadmat, savemat

import ROI_image_reader_stitched as ROI
import shutil
import pickle
import ROI_image_reader_stitched

In [ ]:
import keras
keras.__version__

In [ ]:
home_path = 'F:/IFCB/'
out_folder_of_images = home_path + 'Training_sets_padded/'  #path to the padded images from the training set
out_folder_of_images_unpadded = home_path + 'Training_sets_unpadded/' #path to the unpadded images from the training set
folder_of_images_validation = home_path + 'validation_sets/' #where to put the validation sets

number_of_categories = 112

In [ ]:
def eliminate_background(image):
    shape = image.shape
    mid = int(shape[0] / 2)
    bkgd_mean = image[mid,:].mean()
    bkgd_std = image[mid, :].std()
    image -= bkgd_mean
    image /= (bkgd_std+0.001)
    
    image *= -1
    return image

In [ ]:
#image size
image_size = 300  #an X by X size square

In [ ]:
#move validation set back into main set
photos = os.walk(folder_of_images_validation)

for files in photos:
    print(files[0])
    outdir = files[0].split('/')[-1]
    for picture in files[2]:
        if picture[-3:] == 'png' or picture[-3:] == 'tif':
            #print(picture)
            shutil.move(files[0]+'/'+picture, out_folder_of_images + outdir + '/' + picture)

In [ ]:
#create a validation dataset
import shutil
photos = os.walk(out_folder_of_images)

num_photo = 0
for files in photos:
    print(files[0])
    outdir = files[0].split('/')[-1]
    if outdir not in os.listdir(folder_of_images_validation) and outdir != '':
        os.mkdir('{0}/{1}'.format(folder_of_images_validation, outdir))
    for picture in files[2][0::5]:
        if picture[-3:] == 'png' or picture[-3:] == 'tif':
            #print(picture)
            num_photo += 1
            shutil.move(files[0]+'/'+picture, folder_of_images_validation + outdir + '/' + picture)

In [ ]:
#training data generator
#adding some modifications to allow for data augmentation (images are manipulated some to make more instances of training data)
input_photos = keras_image.ImageDataGenerator(#rotation_range=10,  #allow images to be rotated randomly between 0 and 90 degrees
                                        width_shift_range=5, #randomly shift image this fraction of total width
                                        #height_shift_range=5, #randomly shift image this fraction of total height
                                        horizontal_flip=True, #flip images horizontally
                                        vertical_flip=True, #flip images vertically
                                        fill_mode='nearest', #how to fill in empty space after shift/rotation
                                        #cval=128, #fill value for fill_mode
                                        #samplewise_std_normalization=True,
                                        preprocessing_function=eliminate_background,
                                        rescale = 1/255.,
                                        #zoom_range = 0.1,
                                        #featurewise_center=True,
                                        #featurewise_std_normalization=True
                                       )

#start the actual flow of images for training
photos = input_photos.flow_from_directory(out_folder_of_images, 
                                          shuffle=False,
                                          color_mode='grayscale', #all ifcb images are grayscale
                                          class_mode='categorical', #there are multiple classes of images (i.e. > 2)
                                          target_size=(image_size,image_size),  #squish/stretch images to this size
                                          batch_size=16,) #how many images per batch
                                          

In [ ]:
#training data generator
#adding some modifications to allow for data augmentation (images are manipulated some to make more instances of training data)
input_photos_unpadded = keras_image.ImageDataGenerator(#rotation_range=10,  #allow images to be rotated randomly between 0 and 90 degrees
                                        width_shift_range=5, #randomly shift image this fraction of total width
                                        #height_shift_range=5, #randomly shift image this fraction of total height
                                        horizontal_flip=True, #flip images horizontally
                                        vertical_flip=True, #flip images vertically
                                        fill_mode='nearest', #how to fill in empty space after shift/rotation
                                        #cval=128, #fill value for fill_mode
                                        #samplewise_std_normalization=True,
                                        preprocessing_function=eliminate_background,
                                        rescale = 1/255.,
                                        #zoom_range = 0.1,
                                        #featurewise_center=True,
                                        #featurewise_std_normalization=True
                                       )

#start the actual flow of images for training
photos_unpadded = input_photos_unpadded.flow_from_directory(out_folder_of_images_unpadded, 
                                          shuffle=False,
                                          color_mode='grayscale', #all ifcb images are grayscale
                                          class_mode='categorical', #there are multiple classes of images (i.e. > 2)
                                          target_size=(image_size,image_size),  #squish/stretch images to this size
                                          batch_size=16,) #how many images per batch
                                          

In [ ]:
#validation data generator

#folder_of_images_validation = '/home/campbelllab/IFCB/IFCB_Conv_NN/validation_sets_size_300/'  #images for training

#adding some modifications to allow for data augmentation (images are manipulated some to make more instances of training data)
input_photos_validation = keras_image.ImageDataGenerator(#rotation_range=10,  #allow images to be rotated randomly between 0 and 90 degrees
                                        #width_shift_range=5, #randomly shift image this fraction of total width
                                        #height_shift_range=5, #randomly shift image this fraction of total height
                                        horizontal_flip=True, #flip images horizontally
                                        vertical_flip=True, #flip images vertically
                                        fill_mode='nearest', #how to fill in empty space after shift/rotation[constant, wrap, reflect, nearest]
                                        #cval=128, #fill value for fill_mode
                                        preprocessing_function=eliminate_background,
                                        rescale = 1/255.,
                                        #zoom_range = 0.1,
                                        
                                        )
#start the actual flow of images for training
photos_validation = input_photos_validation.flow_from_directory(folder_of_images_validation, 
                                          #shuffle=False,
                                          color_mode='grayscale', #all ifcb images are grayscale
                                          class_mode='categorical', #there are multiple classes of images (i.e. > 2)
                                          target_size=(image_size,image_size),  #squish/stretch images to this size
                                          #save_to_dir='D:/Python27/Projects/Classifiers/augmented_data/',
                                          batch_size=16) #how many images per batch

In [ ]:
#confusion_matrix data generator

#adding some modifications to allow for data augmentation (images are manipulated some to make more instances of training data)
input_photos_confusion = keras_image.ImageDataGenerator(#rotation_range=10.,  #allow images to be rotated randomly between 0 and 90 degrees
                                        #width_shift_range=5, #randomly shift image this fraction of total width
                                        #height_shift_range=5, #randomly shift image this fraction of total height
                                        #horizontal_flip=True, #flip images horizontally
                                        #vertical_flip=True, #flip images vertically
                                        fill_mode='nearest', #how to fill in empty space after shift/rotation[constant, wrap, reflect, nearest]
                                        #cval=128, #fill value for fill_mode
                                        preprocessing_function=eliminate_background,
                                        rescale = 1/255.,
                                        #zoom_range = 0.15,
                                        )
#start the actual flow of images for training
photos_confusion = input_photos_confusion.flow_from_directory(out_folder_of_images, 
                                          shuffle=False,
                                          color_mode='grayscale', #all ifcb images are grayscale
                                          class_mode='categorical', #there are multiple classes of images (i.e. > 2)
                                          target_size=(image_size,image_size),  #squish/stretch images to this size
                                          batch_size=16 #how many images per batch
                                        )

#Okay the images are all ready to be loaded and will be resized to a 300x300 image (I can change this in the #ImageDataGenerator).

In [ ]:
num_images = len(photos.classes)
num_images

In [ ]:
#try this function to correct for imbalanced classes 
#downloaded this function code from: https://github.com/cbaziotis/keras-utilities/blob/master/kutilities/helpers/data_preparation.py
def get_class_weights(y, smooth_factor=0):
    """
    Returns the weights for each class based on the frequencies of the samples
    :param smooth_factor: factor that smooths extremely uneven weights
    :param y: list of true labels (the labels must be hashable)
    :return: dictionary with the weight for each class
    """
    counter = Counter(y)

    if smooth_factor > 0:
        p = max(counter.values()) * smooth_factor
        for k in counter.keys():
            counter[k] += p

    majority = max(counter.values())

    return {cls: float(majority) / count for cls, count in counter.items()}

In [ ]:
temp_class_weights = get_class_weights(photos.classes, 0.1)

In [ ]:
#start the training of the ensemble model

In [ ]:
#move validation set back into main set
photos = os.walk(folder_of_images_validation)

for files in photos:
    print(files[0])
    outdir = files[0].split('/')[-1]
    for picture in files[2]:
        if picture[-3:] == 'png' or picture[-3:] == 'tif':
            #print(picture)
            shutil.move(files[0]+'/'+picture, out_folder_of_images + outdir + '/' + picture)
            


In [ ]:
path_to_models = '/path/to/models/'

In [ ]:
#load all three models if possible
#combine them into one single ensemble model
#make each individual model untrainable
#train a final layer for weighting the individual models

model1 = load_model(path_to_models + 'CNN_model_mdl1_padded.mdl')
model1.trainable = False

model2 = load_model(path_to_models + 'CNN_model_mdl2_padded.mdl')
model2.trainable = False

model3 = load_model(path_to_models + 'CNN_model_mdl3_padded.mdl')
model3.trainable = False

model4 = load_model(path_to_models + 'CNN_model_mdl1_unpadded.mdl')
model4.trainable = False

model5 = load_model(path_to_models + 'CNN_model_mdl2_unpadded.mdl')
model5.trainable = False

model6 = load_model(path_to_models + 'CNN_model_mdl3_unpadded.mdl')
model6.trainable = False

In [ ]:
#rename the model layers so they don't conflict with each other

for num, layer in enumerate(model1.layers):
    layer.name = 'Model1_'+str(num)
model1.name = 'Model1'

for num, layer in enumerate(model2.layers):
    layer.name = 'Model2_'+str(num)
model2.name =  'Model2'

for num, layer in enumerate(model3.layers):
    layer.name = 'Model3_'+str(num)
model3.name = 'Model3'

for num, layer in enumerate(model4.layers):
    layer.name = 'Model4_'+str(num)
model4.name = 'Model4'

for num, layer in enumerate(model5.layers):
    layer.name = 'Model5_'+str(num)
model5.name =  'Model5'

for num, layer in enumerate(model6.layers):
    layer.name = 'Model6_'+str(num)
model6.name = 'Model6'

In [ ]:
padded_inputs = Input((image_size, image_size, 1))
unpadded_inputs = Input((image_size, image_size, 1))

out1 = model1(padded_inputs)
out2 = model2(padded_inputs)
out3 = model3(padded_inputs)
out4 = model4(unpadded_inputs)
out5 = model5(unpadded_inputs)
out6 = model6(unpadded_inputs)

test = Concatenate()([out1, out2, out3, out4, out5, out6])
test = Dense(112, name='model_ensemble_3')(test)
test = Activation('softmax', name='model_ensemble_4')(test)

ensemble = Model([padded_inputs, unpadded_inputs], test)


In [ ]:
adam = Adam(lr=0.00001, decay=.000001)
ensemble.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'],)

In [ ]:
ensemble.summary()

In [ ]:
photos.reset()
photos_unpadded.reset()
num_images = len(photos.classes)
temp_class_weights = get_class_weights(photos.classes, 0.1)

num_images

In [ ]:
def get_image_batches(num_batches=6795):
    padded = []
    unpadded = []
    answers = []
    for x in range(num_batches):
        if x % 50 == 0:
            print(x, end=',')
        temp1 = photos.next()
        temp2 = photos_unpadded.next()
        padded.extend(temp1[0])
        unpadded.extend(temp2[0])
        answers.extend(temp1[1])
    
    return [padded, unpadded, answers]

In [ ]:
def get_image_batch_generator():
    
    while 1:
        temp1 = photos.next()
        temp2 = photos_unpadded.next()
        
    
        yield [[temp1[0], temp2[0]], temp1[1]]
        
    return

def get_image_batch_generator_prediction():
    
    while 1:
        temp1 = photos.next()
        temp2 = photos_unpadded.next()
        
    
        yield [temp1[0], temp2[0]]
        
    return

In [ ]:
in_padded, in_unpadded, answers = get_image_batches()

In [ ]:
hist = ensemble.fit_generator(get_image_batch_generator(),
                    steps_per_epoch=int(num_images/16),
                    epochs=3,
                    initial_epoch=0,
                    #validation_data = photos_validation,
                    #validation_steps = 600,
                    class_weight=temp_class_weights,  #this is to help with the unbalanced class issue
                          )

In [ ]:
#save the ensemble model
ensemble.save_weights(path_to_models + 'ensemble_model__weights.wts')

In [ ]:
#load the ensemble model
#use this when needed for testing or reloading the model
ensemble.load_weights(path_to_models + 'ensemble_model_weights.wts')

In [ ]:
photos.reset()
photos_unpadded.reset()
num_images = len(photos.classes)
num_images

In [ ]:
photos_confusion.reset()
Y_pred = ensemble.predict_generator(get_image_batch_generator_prediction(), num_images/16 + 1)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
print('Confusion Matrix')
print(confusion_matrix(photos_confusion.classes, y_pred[:num_images]))
check_answer = sort(list(photos_confusion.class_indices))
print('Classification Report')
target_names = check_answer
print(classification_report(photos_confusion.classes, y_pred[:num_images], target_names=target_names))

In [ ]:
conf_mat = pd.DataFrame(confusion_matrix(photos_confusion.classes, y_pred[:num_images]), columns=target_names, index=target_names)
conf_mat['Asterionellopsis'].sum()

In [ ]:
figsize(20, 20)
sns.heatmap(conf_mat.divide(conf_mat.sum()+1), vmax=1,cmap='binary')

In [ ]:
results = [[] for x in range(112)]
photos.reset()
photos_unpadded.reset()

In [ ]:
imgs = get_image_batch_generator()
for z in range(7574):
    ims = next(imgs)
    res = ensemble.predict_on_batch(ims[0])

    for x,y in enumerate(res):
        results[ims[1][x].argmax()].append(res[x][ims[1][x].argmax()])

In [ ]:
for x in range(112):
    temp = np.array(results[x])
    print('Category:', check_answer[x])
    print('avg:', temp.mean())
    print('std:', temp.std())

In [ ]:
thresholds = [np.array(temp).mean() - 3*np.array(temp).std() for temp in results]

In [ ]:
#save the thresholds to a file so that they can be loaded later
with open('/path/to/models/thresholds_ensemble.pck', 'wb') as f:
    pickle.dump(thresholds, f)

In [ ]:
#save the probability results in case we want to change our threshold later
with open('/path/to/models/prob_scores_ensemble.pck', 'wb') as f:
    pickle.dump(results, f)
    

In [ ]:
#load the probability results in case we want to change our threshold later
with open('/path/to/models/thresholds_ensemble.pck', 'rb') as f:
    thresholds = pickle.load(f)
    

In [ ]:
[(check_answer[x], thresholds[x]) for x in range(112)]